## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,23.40,98,100,4.47,light snow
1,1,Codrington,AU,-38.2667,141.9667,75.36,30,78,14.54,broken clouds
2,2,Broome,US,42.2506,-75.8330,43.86,49,1,4.00,clear sky
3,3,Ribeira Grande,PT,38.5167,-28.7000,55.80,58,20,8.05,few clouds
4,4,Constitucion,CL,-35.3333,-72.4167,60.49,56,0,19.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temperature_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                            (city_data_df["Max Temp"] >= min_temp)]
preferred_temperature_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Codrington,AU,-38.2667,141.9667,75.36,30,78,14.54,broken clouds
5,5,Rio Grande,BR,-32.0350,-52.0986,78.91,72,0,15.77,clear sky
7,7,Port Blair,IN,11.6667,92.7500,79.41,74,69,12.66,broken clouds
11,11,Vanimo,PG,-2.6741,141.3028,78.85,87,100,5.39,overcast clouds
15,15,Chuy,UY,-33.6971,-53.4616,73.83,66,0,14.79,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temperature_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_temperatures_df = preferred_temperature_df.dropna()
new_temperatures_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Codrington,AU,-38.2667,141.9667,75.36,30,78,14.54,broken clouds
5,5,Rio Grande,BR,-32.0350,-52.0986,78.91,72,0,15.77,clear sky
7,7,Port Blair,IN,11.6667,92.7500,79.41,74,69,12.66,broken clouds
11,11,Vanimo,PG,-2.6741,141.3028,78.85,87,100,5.39,overcast clouds
15,15,Chuy,UY,-33.6971,-53.4616,73.83,66,0,14.79,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_temperatures_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Codrington,AU,75.36,broken clouds,-38.2667,141.9667,
5,Rio Grande,BR,78.91,clear sky,-32.0350,-52.0986,
7,Port Blair,IN,79.41,broken clouds,11.6667,92.7500,
11,Vanimo,PG,78.85,overcast clouds,-2.6741,141.3028,
15,Chuy,UY,73.83,clear sky,-33.6971,-53.4616,
18,Kapaa,US,79.12,few clouds,22.0752,-159.3190,
19,Rikitea,PF,77.40,light rain,-23.1203,-134.9692,
23,Auki,SB,81.09,heavy intensity rain,-8.7676,160.7034,
24,Georgetown,MY,79.90,scattered clouds,5.4112,100.3354,
25,Mahebourg,MU,74.25,overcast clouds,-20.4081,57.7000,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Codrington,AU,75.36,broken clouds,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
5,Rio Grande,BR,78.91,clear sky,-32.0350,-52.0986,Hotel Atlântico Rio Grande
7,Port Blair,IN,79.41,broken clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
11,Vanimo,PG,78.85,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
15,Chuy,UY,73.83,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
...,...,...,...,...,...,...,...
644,Bonoua,CI,83.82,broken clouds,5.2741,-3.5950,Hotel Bon Samaritian
652,Ati,TD,79.02,overcast clouds,13.2154,18.3353,
653,Ahipara,NZ,70.48,moderate rain,-35.1667,173.1667,Ahipara Holiday Park
657,Nautla,MX,71.01,scattered clouds,20.2167,-96.7833,Casa Blanca


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Codrington,AU,75.36,broken clouds,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
5,Rio Grande,BR,78.91,clear sky,-32.0350,-52.0986,Hotel Atlântico Rio Grande
7,Port Blair,IN,79.41,broken clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
11,Vanimo,PG,78.85,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
15,Chuy,UY,73.83,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
...,...,...,...,...,...,...,...
643,Mozarlandia,BR,77.76,overcast clouds,-14.7447,-50.5706,Hotel Santa Mônica
644,Bonoua,CI,83.82,broken clouds,5.2741,-3.5950,Hotel Bon Samaritian
653,Ahipara,NZ,70.48,moderate rain,-35.1667,173.1667,Ahipara Holiday Park
657,Nautla,MX,71.01,scattered clouds,20.2167,-96.7833,Casa Blanca


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))